<a href="https://colab.research.google.com/github/Sankalpa0011/LLM-Conversational-RAG-App-LangChain-and-HuggingFace./blob/main/LLM_Conversational_RAG_App_LangChain_and_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install the necessary packages
!pip install torch -q
!pip install transfomers -q
!pip install numpy -q
!pip install langchain -q
!pip install langchain_community -q
!pip install langchain-chroma -q
!pip install sentence_transformers -q  # Embedding model

ERROR: Could not find a version that satisfies the requirement transfomers (from versions: none)
ERROR: No matching distribution found for transfomers


## **Initialize HuggingFace LLM**

In [3]:
import os
from google.colab import userdata
from langchain_community.llms import HuggingFaceHub

# Set HuggingFace API key from userdata or environment
os.environ["HUGGING_FACE_TOKEN"] = userdata.get("HUGGING_FACE_TOKEN")

# Access the token from the environment
hf_api_token = os.getenv("HUGGING_FACE_TOKEN")

# Initialize the HuggingFace llm
llm = HuggingFaceHub(
    repo_id = "mistralai/Mistral-7B-v0.1",
    model_kwargs = {"temperature": 0.1, "max_length": 500},
    huggingfacehub_api_token = hf_api_token
)

<ipython-input-3-1e384b961190>:12: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


## **Initialize Embedding Model**

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize the embedding model
embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

<ipython-input-4-7cd10707740a>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## **Initialize Output Parser**

In [5]:
from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()

## **Load PDF Document**

In [6]:
!pip install pypdf -qU

In [7]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader("/content/codeprolk.pdf")

docs = loader.load()

In [8]:
docs

[Document(metadata={'source': '/content/codeprolk.pdf', 'page': 0}, page_content="Introduction to CodePRO LK  \nCodePRO LK  is a dynamic educational platform that offers a diverse range of technology -\nrelated courses in Sinhala, aimed at empowering Sri Lankans with valuable skills in \nprogramming, data science, and machine learning. Founded by Dinesh Piyasamara  during the \nCOVID -19 pandemic, CodePRO LK addresses the growing need for accessible, high -quality \ntech education tailored to the local community.  \n \nFounding and Vision  \nOrigin and Motivation  \nThe inception of CodePRO LK was driven by the challenges posed by the COVID -19 pandemic, \nwhich highlighted the necessity for remote learning and digital skills. Recognizing this, Dinesh \nPiyasamara launched CodePRO LK to provide Sri Lankan students with the  tools and knowledge \nto thrive in a digitally -driven world, all through their native language.  \nVision and Mission  \n• Vision : To assist talented Sri Lankans 

In [9]:
len(docs)

4

In [10]:
docs[0]

Document(metadata={'source': '/content/codeprolk.pdf', 'page': 0}, page_content="Introduction to CodePRO LK  \nCodePRO LK  is a dynamic educational platform that offers a diverse range of technology -\nrelated courses in Sinhala, aimed at empowering Sri Lankans with valuable skills in \nprogramming, data science, and machine learning. Founded by Dinesh Piyasamara  during the \nCOVID -19 pandemic, CodePRO LK addresses the growing need for accessible, high -quality \ntech education tailored to the local community.  \n \nFounding and Vision  \nOrigin and Motivation  \nThe inception of CodePRO LK was driven by the challenges posed by the COVID -19 pandemic, \nwhich highlighted the necessity for remote learning and digital skills. Recognizing this, Dinesh \nPiyasamara launched CodePRO LK to provide Sri Lankan students with the  tools and knowledge \nto thrive in a digitally -driven world, all through their native language.  \nVision and Mission  \n• Vision : To assist talented Sri Lankans i

## **Split Documents Into Chunks**

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap = 50)

# Split the documents into chunks
chunks = text_splitter.split_documents(docs)

In [12]:
len(chunks)

20

## **Create Vector Store And Retriever**

In [13]:
from langchain_chroma import Chroma

# Create the vector store from the document chunks
vector_store = Chroma.from_documents(documents=chunks, embedding=embedding_model)

In [14]:
# Create the retriever from the cvector store
retriever = vector_store.as_retriever()

## **Define Prompt Template**

In [15]:
from langchain.prompts import ChatPromptTemplate

# Define the prompt template
template = """
Answer this question using the provided context only.

{question}

Context:
{context}

Answer:
"""

# Create the prompt template
prompt = ChatPromptTemplate.from_template(template)

In [16]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer this question using the provided context only.\n\n{question}\n\nContext:\n{context}\n\nAnswer:\n'), additional_kwargs={})])

## **Chain Retriever And Prompt Template With LLM**

In [17]:
from langchain.schema.runnable import RunnablePassthrough

# Create the chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

## **Invoke RAG Chain With Example Question**

In [18]:
response = chain.invoke(input="Who is CodeProLK?")
print(response)

Human: 
Answer this question using the provided context only.

Who is CodeProLK?

Context:
[Document(metadata={'page': 3, 'source': '/content/codeprolk.pdf'}, page_content='Partnerships and Collaborations  \nCodePRO LK is exploring partnerships with educational institutions, tech companies, and \nindustry experts to enrich its content and provide learners with access to a broader range of \nresources and opportunities. These collaborations aim to bridge the gap be tween education and \nindustry, ensuring that learners are well -prepared for real -world challenges.'), Document(metadata={'page': 1, 'source': '/content/codeprolk.pdf'}, page_content='Community and Support  \nCodePRO LK has cultivated a vibrant community where learners can interact, share insights, and \nsupport each other. Additionally, the platform offers consultation services for personalized \nlearning support.  \n \nCodePRO LK YouTube Channel  \nOverview  \nThe CodePRO LK YouTube Channel  is a crucial extension of the 

In [20]:
response = chain.invoke(input="What are the courses they provided?")
print(response)

Human: 
Answer this question using the provided context only.

What are the courses they provided?

Context:
[Document(metadata={'page': 1, 'source': '/content/codeprolk.pdf'}, page_content='Learning Experience  \nCourse Structure  \nEach course is meticulously structured to provide a holistic learning experience, comprising:  \n• Video Lectures : Detailed tutorials that break down complex concepts.  \n• Quizzes : Interactive quizzes to reinforce learning.  \n• Assignments : Hands -on tasks to apply theoretical knowledge.  \nCommunity and Support'), Document(metadata={'page': 0, 'source': '/content/codeprolk.pdf'}, page_content='Course Offerings  \nVariety and Accessibility  \nCodePRO LK stands out for its wide array of free courses, all presented in Sinhala. The courses \ncater to various proficiency levels, from beginners to intermediates, ensuring that learners of all \nstages can benefit.  \nKey Courses  \n1. Python GUI – Tkinter : This course covers the essentials of creating grap